In [1]:
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint

In [2]:
url_1= 'http://www.acclaimedmusic.net/061024/1948-09art.htm'
artists_1= (pd.read_html(url_1,header=0))[0]["Unnamed: 1"]

pages= np.arange(2,6)
artists_list=[]

for page in pages:
    url_2= f"http://www.acclaimedmusic.net/061024/1948-09art{page}.htm"
    artists_df= (pd.read_html(url_2,header=0))
    artists_list.append(artists_df[0]["Unnamed: 1"])

artists= artists_1.append(artists_list)
artists.dropna(inplace=True)
artists_names= artists.reset_index()
artists_names.drop(artists_names.columns[0], axis=1, inplace=True)
artists_names.rename(columns={"Unnamed: 1": "Artist Name"}, inplace=True)
artists_names

,Artist Name
0,The Beatles
1,The Rolling Stones
2,Bob Dylan
3,David Bowie
4,Led Zeppelin
...,...
995,The Sundays
996,Herbert
997,Stephen Stills
998,Mazzy Star


In [3]:
artists_names["Artist ID"]= ""
artists_names.head()

,Artist Name,Artist ID
0,The Beatles,
1,The Rolling Stones,
2,Bob Dylan,
3,David Bowie,
4,Led Zeppelin,


In [4]:
for index, row in artists_names.iterrows():
    TOKEN= "BQDguFC62TGl0gOSdc7_GajHPQH_7SUSI4YIrqkXb3_dX2nSsCq5hd9xNpc8Pi0nipJFfdSRxGkhM-3I4fO2acBE0WccompjjPPTiFTmykaYure_IPHlfD2jjRQHIwaFlfVVNrpCZ0D1HsNX7qX8t9m7iaCn03wlLsNYgV2aMwX8ZILpkFiap14QHhKuryn6T5llolXhT0pteahtEAgdygp9kSUzHozS0c1P2iQWslhPEsdU_VbpNODFVGDSDgYGHBUDOzuA82Qg4mS7WcpcFKo0yqOxcNH2"
    url = f"https://api.spotify.com/v1/search?q={row['Artist Name']}&type=artist"
    headers = {'Authorization': "Bearer {}".format(TOKEN)}
    r = requests.get(url, headers=headers).json()
    
    try:
        artists_names.loc[index,'Artist ID'] = r['artists']["items"][0]["id"]
    except:
        
        #Printing an explanation message
        print(f"ID not founded for {row['Artist Name']}")
        artists_names.loc[index,'Artist Name'] = np.NaN
        artists_names.loc[index,'Artist ID'] = np.NaN
        pass
        
artists_names.dropna(inplace=True)
artists_names


ID not founded for Frank Zappa/The Mothers of Invention
ID not founded for Janis Joplin/Big Brother and The Holding Company
ID not founded for The Buzzcocks
ID not founded for Paul McCartney and Wings
ID not founded for The Small Faces
ID not founded for The Dead Kennedys
ID not founded for Harold Melvin and The Bluenotes
ID not founded for The Mahavishnu Orchestra
ID not founded for The Nitty Gritty Dirt Band
ID not founded for The Art Ensemble of Chicago
ID not founded for Sam the Sham and The Pharaos
ID not founded for The Screaming Trees
ID not founded for Michael Hurley/The Unholy Modal Rounders
ID not founded for Rythim Is Rythim


,Artist Name,Artist ID
0,The Beatles,3WrFJ7ztbogyGnTHbHJFl2
1,The Rolling Stones,22bE4uQ6baNwSHPVcDxLCe
2,Bob Dylan,74ASZWbe4lXaubB36ztrGX
3,David Bowie,0oSGxfWSnnOXhD2fKuz2Gy
4,Led Zeppelin,36QJpDe2go2KgaRleHCDTp
...,...,...
995,The Sundays,4nlqDmbzFzbNITaqjJv7D7
996,Herbert,5zCaQxjl110XTrm4LQ1CxY
997,Stephen Stills,4WlSvDKaq1PA2Nr7cCIPxX
998,Mazzy Star,37w38cCSGgKLdayTRjna4W


In [5]:
top10_tracks= pd.DataFrame(columns=["Artist","Spotify_ID","Tracks"])
top10_tracks


,Artist,Spotify_ID,Tracks


In [19]:
# list_id= ["3WrFJ7ztbogyGnTHbHJFl2","22bE4uQ6baNwSHPVcDxLCe"]

counter=1
track_names=[]
for index, row in artists_names.iterrows():
    
    TOKEN= "BQCwZoKVR_DPg0ZfmmjlV0nv3H8ywnP2mlNoceSsDhk-2k0LoGJlhlzgP1vFxYMTX4zrm-ukoS2qGKQk_jFhlC-sZ0ZtK6MSX3br_GYesWM5MD45nv5f-5U-PBtPx50J0jS3E-Q8IIXyEzcNGw3cRzopooQNOj3c3uzym2P905DGG604jdwncjrxmXPiSWdoYSiosqDdKfLEPFG5uPqgLkONpDTIkTuZ5F87yoXchhJ-Vh9zVSdbeNCX6E9r0NKliEDTIr39yH6KoVQQwxIFsxe9P1W6zrdi"
    url = f"https://api.spotify.com/v1/artists/{row['Artist ID']}/top-tracks?country=US"
    headers = {'Authorization': "Bearer {}".format(TOKEN)}
    r = requests.get(url, headers=headers).json() 

    track = r['tracks'][:10]

    for tracks in range(len(track)):    
        try:
            top10_tracks.loc[counter,'Tracks'] = track[tracks]["name"]
            top10_tracks.loc[counter,'Artist'] = track[tracks]["album"]["artists"][0]["name"]
            top10_tracks.loc[counter,'Spotify_ID'] = track[tracks]["album"]["artists"][0]["id"]
            counter+=1
        
        except:
            #Printing an explanation message
            pass
top10_tracks    


,Artist,Spotify_ID,Tracks
1,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Here Comes The Sun - Remastered 2009
2,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Come Together - Remastered 2009
3,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Let It Be - Remastered 2009
4,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Yesterday - Remastered 2009
5,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Hey Jude - Remastered 2015
...,...,...,...
9730,Blind Lemon Jefferson,6EsuMifB8c5rm5P2DdjiLe,Future Blues
9731,Blind Lemon Jefferson,6EsuMifB8c5rm5P2DdjiLe,Match Box Blues
9732,Blind Lemon Jefferson,6EsuMifB8c5rm5P2DdjiLe,Rabbit Foot Blues
9733,Blind Lemon Jefferson,6EsuMifB8c5rm5P2DdjiLe,That Black Snake Moan


In [20]:
grouped= top10_tracks.groupby("Artist")
count= grouped["Spotify_ID"].count()
count

Artist
!!!                                             10
$uicideBoy$                                     10
...And You Will Know Us by the Trail of Dead    10
2Pac                                            10
4th & Orange                                     2
                                                ..
case/lang/veirs                                  7
dvsn                                             1
for KING & COUNTRY                               1
k.d. lang                                        5
lovelytheband                                   10
Name: Spotify_ID, Length: 1178, dtype: int64